# Listeners must have arguments "event" and "metadata" with proper types

In [1]:
from morpheus.common.types.event_sourcing.EventMetadata import EventMetadata
from morpheus.project.domain.events.ProjectEvents import ProjectCreatedEvent
from morpheus.common.infrastructure.event_sourcing.EventPublisher import EventPublisher, listen_to


class Invalid:
  pass


class ProjectorInvalid0:
  @listen_to(ProjectCreatedEvent)
  def on_project_created(self):
    print('Created project')


class ProjectorInvalid1:
  @listen_to(ProjectCreatedEvent)
  def on_project_created(self, invalid: ProjectCreatedEvent, metadata: EventMetadata):
    print('Created project')


class ProjectorInvalid2:
  @listen_to(ProjectCreatedEvent)
  def on_project_created(self, event: ProjectCreatedEvent, invalid: EventMetadata):
    print('Created project')


class ProjectorInvalid3:
  @listen_to(ProjectCreatedEvent)
  def on_project_created(self, event: Invalid, metadata: EventMetadata):
    print('Created project')


class ProjectorInvalid4:
  @listen_to(ProjectCreatedEvent)
  def on_project_created(self, event: ProjectCreatedEvent, metadata: Invalid):
    print('Created project')


event_publisher = EventPublisher()
try:
  projector = ProjectorInvalid0()
  event_publisher.register(projector)
except Exception as e:
  print(e)

try:
  projector = ProjectorInvalid1()
  event_publisher.register(projector)
except Exception as e:
  print(e)

try:
  projector = ProjectorInvalid2()
  event_publisher.register(projector)
except Exception as e:
  print(e)

try:
  projector = ProjectorInvalid3()
  event_publisher.register(projector)
except Exception as e:
  print(e)

try:
  projector = ProjectorInvalid4()
  event_publisher.register(projector)
except Exception as e:
  print(e)

Event listener <bound method ProjectorInvalid0.on_project_created of <__main__.ProjectorInvalid0 object at 0x1379aeff0>> must have an argument called "event" that should by type hinted with a subclass of EventBase
Event listener <bound method ProjectorInvalid1.on_project_created of <__main__.ProjectorInvalid1 object at 0x137a21490>> must have an argument called "event" that should by type hinted with a subclass of EventBase
Event listener <bound method ProjectorInvalid2.on_project_created of <__main__.ProjectorInvalid2 object at 0x1379aeff0>> must have an argument called "metadata" that should by type hinted with a subclass of EventMetadata
Event listener <bound method ProjectorInvalid3.on_project_created of <__main__.ProjectorInvalid3 object at 0x137a21490>> must have an argument called "event" that should by type hinted with a subclass of EventBase
Event listener <bound method ProjectorInvalid4.on_project_created of <__main__.ProjectorInvalid4 object at 0x1379aeff0>> must have an arg

# Cannot register same listener twice

In [2]:
class Projector:
  @listen_to(ProjectCreatedEvent)
  def on_project_created(self, event: ProjectCreatedEvent, metadata: EventMetadata):
    print(f'Created project {event.get_project_id()} with metadata {metadata}')


try:
  event_publisher = EventPublisher()
  projector = Projector()
  event_publisher.register(projector)
  event_publisher.register(projector)
except Exception as e:
  print(e)

Event handler already registered


# Create projects

In [3]:
from morpheus.project.infrastructure.persistence.ProjectSummaryRepository import project_summary_repository
from morpheus.project.infrastructure.event_sourcing.ProjectEventStore import project_event_store
from morpheus.project.types.User import UserId
from morpheus.project.types.Project import ProjectId, Name, Description, Tags
from morpheus.project.application.write.ProjectCommandHandlers import CreateProjectCommand, CreateProjectCommandHandler

for i in range(10):
  create_project_command = CreateProjectCommand(
    project_id=ProjectId.new(),
    name=Name.from_str(f'Project {i}'),
    description=Description.from_str(f'Description {i}'),
    tags=Tags.from_list([f'tag{i}', f'tag{i + 1}']),
    created_by=UserId.new()
  )
  CreateProjectCommandHandler.handle(create_project_command)

event_envelopes = project_event_store.get_all_events_ordered_by_version()
print(f'Found {len(event_envelopes)} events')
for event_envelope in event_envelopes:
  print(f'"{event_envelope.get_event().get_event_name().to_str()}" occurred at {event_envelope.event.get_occurred_at().to_iso_with_timezone()}')

project_summaries = project_summary_repository.find_all()
print(f'Found {len(project_summaries)} project summaries')
for project_summary in project_summaries:
  print(f'Project {project_summary.project_id.to_str()}: {project_summary.project_name.to_str()}')

Found 10 events
"Project Created" occurred at 2024-03-21T21:19:32.151659+01:00
"Project Created" occurred at 2024-03-21T21:19:32.167225+01:00
"Project Created" occurred at 2024-03-21T21:19:32.170740+01:00
"Project Created" occurred at 2024-03-21T21:19:32.174530+01:00
"Project Created" occurred at 2024-03-21T21:19:32.180303+01:00
"Project Created" occurred at 2024-03-21T21:19:32.188147+01:00
"Project Created" occurred at 2024-03-21T21:19:32.195263+01:00
"Project Created" occurred at 2024-03-21T21:19:32.198633+01:00
"Project Created" occurred at 2024-03-21T21:19:32.201547+01:00
"Project Created" occurred at 2024-03-21T21:19:32.205303+01:00
Found 10 project summaries
Project ab8387c0-aa3d-4628-a8c9-668340243295: Project 0
Project e7866ffe-bda5-49bc-b4ba-167a6f405961: Project 1
Project bbe8639e-71bb-4f63-9e07-770614315748: Project 2
Project 3bb1c7f8-9543-4d9b-b94d-6e34a6e4edf2: Project 3
Project 41183b3c-7dd6-4985-92bf-e7b4664fbf85: Project 4
Project b61d9b61-9942-45d4-a600-c2ace6a5e257: P

# Reproject project summaries

In [4]:
from morpheus.project.presentation.cli.ProjectionCliCommands import ReprojectProjectSummariesCliCommand

ReprojectProjectSummariesCliCommand.run()

Reprojecting project summaries
------------------------------

Found 10 events
Reset projector
Successfully reprojected project summaries


In [5]:
project_summaries = project_summary_repository.find_all()
print(f'Found {len(project_summaries)} project summaries')
for project_summary in project_summaries:
  print(f'Project {project_summary.project_id.to_str()}: {project_summary.project_name.to_str()}')

Found 10 project summaries
Project ab8387c0-aa3d-4628-a8c9-668340243295: Project 0
Project e7866ffe-bda5-49bc-b4ba-167a6f405961: Project 1
Project bbe8639e-71bb-4f63-9e07-770614315748: Project 2
Project 3bb1c7f8-9543-4d9b-b94d-6e34a6e4edf2: Project 3
Project 41183b3c-7dd6-4985-92bf-e7b4664fbf85: Project 4
Project b61d9b61-9942-45d4-a600-c2ace6a5e257: Project 5
Project 3be28fe7-7937-4e12-aa39-49c5557540f4: Project 6
Project ac40eb43-c9d2-46fd-8bf4-37d8fc837869: Project 7
Project 01cc2722-2a3e-4445-a9b1-a6f274c51c4b: Project 8
Project 3d7241b0-b396-43a9-966d-3baf40cad1fd: Project 9
